<table align="left">
    <tr>
        <td bgcolor="#FFFFFF">
            <img src="img/scca_data_science_logo.png" width="20%" align="left">
        </td>
    </tr>
    <tr>
        <td>
            <p align="left">
                <font size="4">
                    <b>SCCA Data Science</b>
                </font>
            </p> 
        </td>
    </tr>
</table>


<img src="img/kraken.jpeg" width="5%" align="left">

# Project Name

## Overview


## Ad Hoc Request


### Requester
- FirstName Lastname, Credentials
    - email:  
    - role:   
    - office: 
    
### Data Science PM
- Assigned to:
    - FirstName, LastName
    - email: user@seattlecca.org
    - role: Data Scientist

### Deliverables



## Research
 

### Reference


## Jupyter Config

In [ ]:
# Expand cell width to 100% browser width for Jupyter notebook
display("text/html", "<style>.container { width:100% !important; }</style>")

<img src="img/julia.png" width="10%" align="left">

## Julia Environment Configuration

In [ ]:
using Pkg;

In [ ]:
Pkg.activate("../")

In [ ]:
Pkg.status()

In [ ]:
using CSV;
using AWSCore;
using AWSSDK.S3;
using StatsPlots;
using DataFrames;

In [ ]:
# Plotting Constant
const plt = StatsPlots;

In [ ]:
# Set plotting backend
#pyplot();

## Data Acquistion

In [ ]:
# Config AWS Object & Crendentials
aws = aws_config(region="us-west-2");

In [ ]:
# Project S3 Constants
s3_bucket      = "datascience-phi-adhoc";
project_name   = "qa";
s3_inputdir    = "/input/";
s3_outputdir   = "/output/";
s3_processdir  = "/processed/";
s3_output_ext  = "-julia-output";
s3_file        = "datasci-qa";
s3_filetype    = "csv";

In [ ]:
# Read CSV from S3 Input and Load into DataFrame
get_config = Dict{String, String}(
    "Bucket"    => "$(s3_bucket)",
    "Key"       => "$(project_name)$(s3_inputdir)$(s3_file).$(s3_filetype)")
file_content = S3.get_object(aws, get_config);

In [ ]:
# Read from AWS S3 file stream
qa_df = CSV.File(IOBuffer(file_content)) |> DataFrame;

## Data Engineering

## EDA (Exploratory Data Analysis)

In [ ]:
first(qa_df, 3)

In [ ]:
last(qa_df, 3)

In [ ]:
names(qa_df)

In [ ]:
describe(qa_df)

In [ ]:
# Shape of DataFrame
df_size = size(qa_df)
println("Rows: $(df_size[1]) \nColumns: $(df_size[2])")

## Modeling

# Output

In [ ]:
# Convert DataFrame to IO Stream
io = IOBuffer()
CSV.write(io, qa_df; delim='\u2588')
csv_string = String(take!(io));



In [ ]:
# Confirm CSV
show(csv_string)

In [ ]:
# Put CSV into S3 Output with TLS inflight and mandated AES256 at rest
put_config = Dict{String, Any}(
    "Bucket"    => "$(s3_bucket)",
    "Key"       => "$(project_name)$(s3_outputdir)$(s3_file)$(s3_output_ext).$(s3_filetype)",
    "Body"      => csv_string,
    "headers"   => Dict{String, String}(
        "Content-Type"                  => "text/csv",
        "x-amz-server-side-encryption"  => "AES256"))
S3.put_object(aws, put_config);

In [ ]:
# Get CSV Metadata
meta_config = Dict{String, String}(
    "Bucket"    => "$(s3_bucket)",
    "Key"       => "$(project_name)$(s3_outputdir)$(s3_file)$(s3_output_ext).$(s3_filetype)")
metadata = S3.head_object(aws, meta_config)
println("Metadata: $metadata")